In [16]:
from pathlib import Path
from PIL import Image
import numpy as np
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from registrator import Registrator
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm
import cv2

In [26]:
### padding for square-shaped image generation
def fit_to_square_bf(img):
    y, x, _ = img.shape

    # fit to even numbers
    if y%2 != 0:
        img = np.pad(img, [(1, 0), (0, 0), (0, 0)], mode='maximum')
    elif x%2 != 0:
        img = np.pad(img, [(0, 0), (1, 0), (0, 0)], mode='maximum')

    # padding
    pad = abs(y-x) // 2
    img = np.pad(img, ((0, 0), (pad, pad), (0, 0)), mode='maximum') if y > x else np.pad(img, ((pad, pad), (0, 0), (0, 0)), mode='maximum')

    return img


def fit_to_square_ri(img):
    y, x= img.shape

    # fit to even numbers
    if y%2 != 0:
        img = np.pad(img, [(1, 0), (0, 0)], mode='minimum')
    elif x%2 != 0:
        img = np.pad(img, [(0, 0), (1, 0)], mode='minimum')

    # padding
    pad = abs(y-x) // 2
    img = np.pad(img, ((0, 0), (pad, pad)), mode='minimum') if y > x else np.pad(img, ((pad, pad), (0, 0)), mode='minimum')

    return img

### image simplifying
def simplifying(img, type='BF', thres=245):
    if type=='BF':
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        thres = (img < thres).astype(np.uint8)
    elif type=='RI':
        thres = (img > thres).astype(np.uint8)
    kernel = np.ones((3, 3), np.uint8)
    kernel2 = np.ones((6, 6), np.uint8)

    closing = cv2.morphologyEx(thres.astype(np.uint8), cv2.MORPH_CLOSE, kernel, iterations=3)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel2, iterations=1)

    return closing

In [3]:
# Load image
# samples which have stitched images
data_path = Path('/data4/NCC/dataset/NCC_missing')
stitched_ris = sorted(data_path.rglob('*Stitching.TCF'))
stitched_bfs = sorted(data_path.rglob('*Stitching000000BF.PNG'))


# samples which do not have stitched images (single FOV images)
temp = sorted(data_path.glob('*/*P20*'))
single_paths = [x for x in temp 
              if x.is_dir() is True 
              and 'T' not in x.name]
single_ris = [sorted(x.rglob('*.TCF'))[0] for x in single_paths]
single_bfs = [sorted(x.rglob('*BF.PNG'))[0] for x in single_paths]

In [39]:
# Registration Loop
for i, paths in enumerate(tqdm(zip(single_ris, single_bfs), total=len(single_ris))):
    with h5py.File(paths[0], 'r') as f:
        ri = f['Data/2DMIP/000000'][()]
    bf = np.asarray(Image.open(paths[1]))
    
    # resizing
    bf = fit_to_square_bf(bf)
    ri = fit_to_square_ri(ri)
    
    bf = cv2.resize(bf, dsize=ri.shape, interpolation=cv2.INTER_LINEAR)
    bf = cv2.cvtColor(bf, cv2.COLOR_RGB2GRAY)
    bf = np.expand_dims(bf, 2)
    ri = np.expand_dims(ri, 2)
    
    lr = 0.01
    loss = "mse"
    num_epochs = 400
    registrator = Registrator(
        lr=lr, loss=loss, in_chans=1, num_epochs=num_epochs, device=)

  0%|          | 0/50 [00:00<?, ?it/s]

(380, 380)
